In [ ]:
# declaraciones de librerías
using DataStructures

In [ ]:
type gFlows
    nVertices::Int64
    capacidades::Array{Int64,2} #arco[i,j] indica la distancia entre i y j 
end

type arcoFlow
    origen::Int64
    destino::Int64
    capacidad::Int64
end

type gFlowsSparse
    nVertices::Int64
    nArcos::Int64
    arcos::Array{arcoFlow,1} #arco[i,j] indica la distancia entre i y j 
end


In [ ]:
function generarGrafoDense(n::Int64,maxValue::Int64)
    a=Array{Int64}(n,n)
    rand!(a,1:maxValue)
    for i in 1:n
        a[i,i]=0
    end
    for i in 1:n
        for j in i+1:n
            if i!=j
                if randn()<0.0
                    a[i,j]=0.0
                else
                    a[j,i]=0.0
                end
            end
        end
    end
    return gFlows(n,a) 
end

function generarGrafoSparse(n::Int64,sp::Float64,maxValue::Int64)
    nArcos=0
    a=arcoFlow[]
    for i in 1:n
        for j in i+1:n
            if rand() <= 2.0*sp
                nArcos += 1
                if randn() < 0.0
                    push!(a,arcoFlow(i,j,rand(1:maxValue)))
                else
                    push!(a,arcoFlow(j,i,rand(1:maxValue)))
                end
            end
        end
    end
    return gFlowsSparse(n,nArcos,a) 
end

function fromDenseToSparse(g::gFlows)
    a=arcoFlow[]
    nArcos=0
    for i in 1:g.nVertices
        for j in 1:g.nVertices
            if g.capacidades[i,j]>0
                nArcos += 1
                push!(a,arcoFlow(i,j,g.capacidades[i,j]))
            end
        end
    end    
    return gFlowsSparse(g.nVertices,nArcos,a)
end

function fromSparseToDense(g::gFlowsSparse)
    a=zeros(Int64,g.nVertices,g.nVertices)
    fill!(a,0)
    for i in 1:g.nArcos
        a[g.arcos[i].origen,g.arcos[i].destino]=g.arcos[i].capacidad
    end
    return gFlows(g.nVertices,a)
end

# Versión de Edmons y Karp del algoritmo de Ford-Fulkerson

El algoritmo utiliza la idea de red residual. La red residual es aquella por la que puede pasarse flujo (ya sea avanzando flujo por la red o reduciendo flujo por la misma). En la red residual busca un camino (en el caso de Edmons-Karp, un camino con el número mínimo de arcos) y aumenta la cantidad transportada por los arcos del camino hasta saturarlo. Este camino se encuentra mediante un algoritmo tipo Breadth-first

El proceso se repite hasta que no puede pasarse flujo y puede representarse a través del siguiente grafo:

<b>P1. Inicialización:</b> Inicializamos los flujos a 0

<b>P2. Buscar camino:</b> Buscamos un camino en la red residual usando BFS. Si no encontramos camino, Fin.

<b>P3. Aumentamos flujo:</b> Buscamos el flujo máximo por el arco y actualizamos la red residual. Ir al paso 2. 

Vamos a ver tres versiones que se diferencian por el formato utilizado para almacenar y tratar los datos.

In [ ]:
# Versión Dense 

function conectividadBreadthFirst(G::gFlows,flows::Array{Int64,2},origen::Int64,destino::Int64)
    ∞=1000000
    #arrays para construir evaluación
    color=falses(G.nVertices)
    π=Array(Int64,G.nVertices)
    fill!(π,-1)
    #inicializamos para el vertice
    color[origen]=true
    Q = Deque{Int64}()
    push!(Q,origen)
    while isempty(Q)==false
        u=shift!(Q)
        #println("\t",u)
        for i in 1:G.nVertices
            if color[i]==false && flows[u,i]>0
                if i==destino
                    π[i]=u
                    return π
                end
                color[i]=true
                π[i]=u
                push!(Q,i)
            end
        end
    end
    return π
end

function ffDense(g::gFlows,origin::Int64,destination::Int64)
    ∞=1000000
    flow=deepcopy(g.capacidades)
    #println("flow: ",flow)
    maxFlow=0
    π=conectividadBreadthFirst(g,flow,origin,destination)
    while π[destination]!= -1
        #print("pi: ",π)
        extraFlow=∞
        c=destination
        while π[c] != -1
            extraFlow=min(extraFlow,flow[π[c],c])
            c=π[c]
        end
        c=destination
        while π[c] != -1
            flow[π[c],c] -= extraFlow
            flow[c,π[c]] += extraFlow
            c=π[c]
        end
        maxFlow += extraFlow
        #println("\t",extraFlow,"\t",maxFlow)
        π=conectividadBreadthFirst(g,flow,origin,destination)
    end
    return maxFlow,flow
end

In [ ]:
# Versión Mixed sparse (arcos) y dense (flujos)

function conectividadBreadthFirstMixed(n::Int64,flows::Array{Int64,2},pOrigen::Array{Int64,1},destinos::Array{Int64,1},origen::Int64,destino::Int64)
    ∞=1000000
    #arrays para construir evaluación
    color=falses(n)
    π=Array(Int64,n)
    fill!(π,-1)
    #inicializamos para el vertice
    color[origen]=true
    Q = Deque{Int64}()
    push!(Q,origen)
    while isempty(Q)==false
        u=shift!(Q)
        #println("\t",u)
        for ii in pOrigen[u]:pOrigen[u+1]-1
            i=destinos[ii]
            if color[i]==false && flows[u,i]>0
                if i==destino
                    π[i]=u
                    return π
                end
                color[i]=true
                π[i]=u
                push!(Q,i)
            end
        end
    end
    return π
end

function prepararMixed(g::gFlowsSparse)
    #flow cubre la estructura dense de flujos
    flow=zeros(Int64,g.nVertices,g.nVertices)
    for i in 1:g.nArcos
         flow[g.arcos[i].origen,g.arcos[i].destino]=g.arcos[i].capacidad
    end
    #pOrigen y destinos cubre la estructura sparse de arcos
    pOrigen = Int64[]
    destinos = Int64[] 
    nArcos=1
    for i in 1:g.nVertices
        push!(pOrigen,nArcos)
        for nA in 1:g.nArcos
            if g.arcos[nA].origen==i
                push!(destinos,g.arcos[nA].destino)
                nArcos += 1
            end
            if g.arcos[nA].destino==i
                push!(destinos,g.arcos[nA].origen)
                nArcos += 1
            end
        end
    end
    push!(pOrigen,nArcos)
    return flow,pOrigen,destinos
end

function ffMixed(G::gFlowsSparse,
                flows::Array{Int64,2},
                pOrigen::Array{Int64,1},
                destinos::Array{Int64,1},
                origen::Int64, #s
                destino::Int64) #t
    ∞=10000000
    maxFlow=0
    π=conectividadBreadthFirstMixed(g.nVertices,flows,pOrigen,destinos,origen,destino)
    while π[destino]!= -1
        #print("pi: ",π)
        extraFlow=∞
        c=destino
        while π[c] != -1
            extraFlow=min(extraFlow,flow[π[c],c])
            c=π[c]
        end
        c=destino
        while π[c] != -1
            flows[π[c],c] -= extraFlow
            flows[c,π[c]] += extraFlow
            c=π[c]
        end
        maxFlow += extraFlow
        #println("\t",extraFlow,"\t",maxFlow)
        π=conectividadBreadthFirstMixed(g.nVertices,flows,pOrigen,destinos,origen,destino)
    end
    return maxFlow,flows
end

In [ ]:
# Versión sparse (arcos y flujos)

function conectividadBreadthFirstSparse(n::Int64,
                            flows::Array{Int64,1},
                            pOrigen::Array{Int64,1},
                            destinos::Array{Int64,1},
                            edge::Array{Int64,1},
                            origen::Int64,
                            destino::Int64)

    ∞=1000000
    #arrays para construir evaluación
    color=falses(n)
    π=Array(Int64,n)
    fill!(π,-1)
    #inicializamos para el vertice
    color[origen]=true
    Q = Deque{Int64}()
    push!(Q,origen)
    while isempty(Q)==false
        u=shift!(Q)
        #println("\tanalizo:",u)
        for ii in pOrigen[u]:pOrigen[u+1]-1
            #println("\t\conexión en arco: ",ii," destino: ",destinos[ii])
            i=destinos[ii]
            if color[i]==false && flows[edge[ii]]>0 #hemos cambiado aqui los flows.
                if i==destino
                    π[i]=u
                    return π
                end
                color[i]=true
                π[i]=u
                push!(Q,i)
            end
        end
    end
    return π
end

function conectividadBreadthFirstSparse(n::Int64,
                            flows::Array{Int64,1},
                            pOrigen::Array{Int64,1},
                            destinos::Array{Int64,1},
                            edge::Array{Int64,1},
                            color::Array{Bool,1},
                            π::Array{Int64,1},
                            origen::Int64,
                            destino::Int64)

    ∞=1000000
    #arrays para construir evaluación
    color=falses(n)
    fill!(π,-1)
    #inicializamos para el vertice
    color[origen]=true
    Q = Deque{Int64}()
    push!(Q,origen)
    while isempty(Q)==false
        u=shift!(Q)
        #println("\tanalizo:",u)
        for ii in pOrigen[u]:pOrigen[u+1]-1
            #println("\t\conexión en arco: ",ii," destino: ",destinos[ii])
            i=destinos[ii]
            if color[i]==false && flows[edge[ii]]>0 #hemos cambiado aqui los flows.
                if i==destino
                    π[i]=u
                    return π
                end
                color[i]=true
                π[i]=u
                push!(Q,i)
            end
        end
    end
    return π
end

function prepararSparse(g::gFlowsSparse)
    #flows, edge y oppositeEdge cubren la estructura de flujos
    flows=Int64[] 
    for i in 1:g.nArcos
        push!(flows,g.arcos[i].capacidad) #el arco original es el +
        push!(flows,0) #el arco opposite es el -
    end
    #pOrigen y destinos cubre la estructura sparse de arcos
    pOrigen = Int64[]
    destinos = Int64[] 
    edge = Int64[]
    edgeOpposite = Int64[]
    nArcos=1
    for i in 1:g.nVertices
        push!(pOrigen,nArcos)
        for nA in 1:g.nArcos
            if g.arcos[nA].origen==i
                push!(destinos,g.arcos[nA].destino)
                push!(edge,(nA-1)*2+1)
                push!(edgeOpposite,nA*2)
                nArcos += 1
            end
            if g.arcos[nA].destino==i
                push!(destinos,g.arcos[nA].origen)
                push!(edge,nA*2)
                push!(edgeOpposite,(nA-1)*2+1)
                nArcos += 1
            end
        end
    end
    push!(pOrigen,nArcos)
    return flows,pOrigen,destinos,edge,edgeOpposite
end

function whichOne(pOrigen::Array{Int64,1},destinos::Array{Int64,1},or::Int64,des::Int64)
    for i in pOrigen[or]:pOrigen[or+1]-1
        if des==destinos[i]
            return i
        end
    end
    println("error\n")
    return 0
end

function ffSparse(G::gFlowsSparse,
                flows::Array{Int64,1},
                pOrigen::Array{Int64,1},
                destinos::Array{Int64,1},
                edge::Array{Int64,1},
                edgeOpposite::Array{Int64,1},
                origen::Int64, #s
                destino::Int64) #t
    ∞=10000000
    maxFlow=0
    π=conectividadBreadthFirstSparse(G.nVertices,flows,pOrigen,destinos,edge,origen,destino)
    while π[destino]!= -1
        #print("pi: ",π,"\n")
        extraFlow=∞
        c=destino
        while π[c] != -1
            arcNum=whichOne(pOrigen,destinos,π[c],c)
            #println("\t\tarcNum sale como: ",arcNum," y hay: ",size(flows))
            extraFlow=min(extraFlow,flows[edge[arcNum]])
            c=π[c]
        end
        c=destino
        while π[c] != -1
            arcNum=whichOne(pOrigen,destinos,π[c],c)
            flow[edge[arcNum]] -= extraFlow
            flow[edgeOpposite[arcNum]] += extraFlow
            c=π[c]
        end
        maxFlow += extraFlow
        #println("\t",extraFlow,"\t",maxFlow)
        π=conectividadBreadthFirstSparse(G.nVertices,flows,pOrigen,destinos,edge,origen,destino)
    end
    return maxFlow,flow
end

function ffSparseAlternativo(G::gFlowsSparse,
                flows::Array{Int64,1},
                pOrigen::Array{Int64,1},
                destinos::Array{Int64,1},
                edge::Array{Int64,1},
                edgeOpposite::Array{Int64,1},
                origen::Int64, #s
                destino::Int64) #t
    ∞=10000000
    maxFlow=0
    π=zeros(Int64,G.nVertices)
    color=falses(G.nVertices)
    π=conectividadBreadthFirstSparseAlternativo(G.nVertices,flows,pOrigen,destinos,edge,color,π,origen,destino)
    while π[destino]!= -1
        #print("pi: ",π,"\n")
        extraFlow=∞
        c=destino
        while π[c] != -1
            arcNum=whichOne(pOrigen,destinos,π[c],c)
            #println("\t\tarcNum sale como: ",arcNum," y hay: ",size(flows))
            extraFlow=min(extraFlow,flows[edge[arcNum]])
            c=π[c]
        end
        c=destino
        while π[c] != -1
            arcNum=whichOne(pOrigen,destinos,π[c],c)
            flow[edge[arcNum]] -= extraFlow
            flow[edgeOpposite[arcNum]] += extraFlow
            c=π[c]
        end
        maxFlow += extraFlow
        #println("\t",extraFlow,"\t",maxFlow)
        π=conectividadBreadthFirstSparseAlternativo(G.nVertices,flows,pOrigen,destinos,edge,origen,destino,π)
    end
    return maxFlow,flow
end

In [ ]:
#g=generarGrafoDense(10,100)
#gS=fromDenseToSparse(g)
gS=generarGrafoSparse(5000,0.1,1000)
g=fromSparseToDense(gS)

#maxFlow,flows=ffDense(g,1,2)
#println("flujo (1): ",maxFlow)
#flow,pOrigen,destinos=prepararMixed(gS)
#maxFlowM,flowsM=ffMixed(gS,flow,pOrigen,destinos,1,2)
#println("flujo (2): ",maxFlowM)
#flow,pOrigen,destinos,edge,edgeOpposite=prepararSparse(gS)
#maxFlowS,flowsS=ffSparse(gS,flow,pOrigen,destinos,edge,edgeOpposite,1,2)
#println("flujo (3): ",maxFlowS)

@time(ffDense(g,1,2))
flow,pOrigen,destinos=prepararMixed(gS)
@time(ffMixed(gS,flow,pOrigen,destinos,1,2))
flow,pOrigen,destinos,edge,edgeOpposite=prepararSparse(gS)
@time(ffSparse(gS,flow,pOrigen,destinos,edge,edgeOpposite,1,2))
@time(ffSparseAlternativo(gS,flow,pOrigen,destinos,edge,edgeOpposite,1,2))


## Conclusiones y comentarios

Como podemos observar existen diferencias significativas entre diferentes implementaciones y en algún caso hay algunos "tradeoffs".

Es más, algunas implementaciones requieren tiempos extras.

# Algoritmo de Dinic

Dinic intenta evitar la fase de construcción del camino aumentante aprovechando todos los caminos posibles de una longitud antes de pasar a la siguiente. 

El algoritmo queda como sigue:

<b>P1. Inicialización:</b> Inicializamos los flujos a 0

<b>P2. Construcción grafo nivel:</b> Construimos un grafo de nivel en el grafo residual usando BFS.

<b>P3. Aumentos flujo:</b> Buscamos caminos que aumenten el usando DFS hasta que no existan más caminos aumentantes en el grafo de nivel.

<b>P4. Aumentamos flujo:</b> Aplicamos todos los caminos encontrados y volvemos al paso 2. 

Este procedimiento lo vamos a implementar con una estructura un poco diferente. Obviamente se podría usar exactamente las mismas estructuras expuestas, pero el objetivo aquí es ver otras técnicas.

In [ ]:
type Edge
    v::Int64    # endpoint 
    flow::Int64 # flow
    c::Int64    # capacity
    rev::Int64  # index of reverse edge
end

type graph
    n::Int64        # Number of nodes
    level::Array{Int64,1} # level of the nodes
    nArcos::Array{Int64,1}
    adj::Array{Edge,2}  #adjacency
end

In [ ]:
function fromSparseToGraph(g::gFlowsSparse)
    level=zeros(Int64,g.nVertices)
    nArcos=zeros(Int64,g.nVertices)
    for i in 1:g.nArcos
        nArcos[g.arcos[i].origen] += 1
        nArcos[g.arcos[i].destino] += 1
    end
    adj= Array{Edge}(g.nVertices,maximum(nArcos))
    nArcos=zeros(Int64,g.nVertices)
    for i in 1:g.nArcos
        u=g.arcos[i].origen
        v=g.arcos[i].destino
        nArcos[u] += 1
        nArcos[v] += 1
        adj[u,nArcos[u]]=Edge(v,0,g.arcos[i].capacidad,nArcos[v])
        adj[v,nArcos[v]]=Edge(u,0,0,nArcos[u])
    end
    return graph(g.nVertices,level,nArcos,adj)
end

#encuentra el label del grafo mediante BFS y verifica si existe flujo
function DinicBFS(g::graph,s::Int64,t::Int64)
    fill!(g.level,-1)
    g.level[s]=0
    #dequeue para breadth first search
    Q = Deque{Int64}()
    push!(Q,s)
    while isempty(Q)==false
        u=shift!(Q)
        for i in 1:g.nArcos[u]
            if g.level[g.adj[u,i].v]<0 && g.adj[u,i].flow<g.adj[u,i].c
                g.level[g.adj[u,i].v]=g.level[u]+1
                push!(Q,g.adj[u,i].v)
            end
        end
    end
    if g.level[t]<0 
        return false
    else
        return true
    end
end

function DinicDFSVisit(G::graph,u::Int64,t::Int64,color::BitArray{1},π::Array{Int64,1},arco::Array{Int64,1})
    if π[t]> 0
        return
    end
    for c in 1:G.nArcos[u]
        i=G.adj[u,c].v
        if (color[i]==false) && (G.level[u]==G.level[i]-1) && (G.adj[u,c].flow<G.adj[u,c].c)
            #println(" marco ",i," con ",G.adj[u,c].flow,"\t",G.adj[u,c].c)
            color[i]=true
            π[i]=u
            arco[i]=c
            DinicDFSVisit(G,i,t,color,π,arco)
        end
    end
end

function DinicDFS(G::graph,s::Int64,t::Int64)
    ∞=100000000
    color=falses(G.n)
    color[s]=true
    arco=zeros(Int64,G.n)
    π=zeros(Int64,G.n)
    π[s]= (-1)
    for c in 1:G.nArcos[s]
        i=G.adj[s,c].v
        #println("\t",c," en $s .. ",G.nArcos[s]," .. testeando $i ",color[i],"\tlevel: ",G.level[s]," . ",G.level[i],"\t",G.adj[s,c].flow," . ",G.adj[s,c].c)
        if (color[i]==false) && (G.level[s]==G.level[i]-1) && (G.adj[s,c].flow<G.adj[s,c].c)
            #println("*marco ",i," con ",G.adj[s,c].flow,"\t",G.adj[s,c].c)
            color[i]=true
            π[i]=s
            arco[i]=c
            DinicDFSVisit(G, i,t,color,π,arco)
        end
    end
    #calcular flow y aumentar
    if π[t] == 0
        return 0
    end
    extraFlow=∞
    c=t
    while π[c] != -1
        extraFlow=min(extraFlow,G.adj[π[c],arco[c]].c-G.adj[π[c],arco[c]].flow)
        c=π[c]
    end
    c=t
    while π[c] != -1
        G.adj[π[c],arco[c]].flow += extraFlow
        G.adj[ G.adj[π[c],arco[c]].v , G.adj[π[c],arco[c]].rev ].flow -= extraFlow;
        c=π[c]
    end
    #println("extraFlow: ",extraFlow)
    return extraFlow
end

function DinicMaxFlow(g::graph,s::Int64,t::Int64)
    maxFlow=0
    while DinicBFS(g,s,t) == true
        while true
            flow = DinicDFS(g,s,t)
            if flow>0
                maxFlow += flow
            else
                break
            end
        end
    end
    return maxFlow
end

                        

In [ ]:
gS=generarGrafoSparse(500,0.1,1000)
gD=fromSparseToGraph(gS)

maxFlow=DinicMaxFlow(gD,1,2)
println("dinic: ",maxFlow)
flow,pOrigen,destinos,edge,edgeOpposite=prepararSparse(gS)
maxFlowS,flowsS=ffSparse(gS,flow,pOrigen,destinos,edge,edgeOpposite,1,2)
println("flujo (3): ",maxFlowS)


# Algoritmo de Goldberg y Tarjan (Push-relabel)

El algoritmo de Goldberg y Tarjan se basa en una idea diferente a la del resto de algoritmos mostrados con anterioridad. Mientras mantienen soluciones factibles primales y aumentan el flujo, el algoritmo de Goldberg y Tarjan permite la infactibilidad primal. Para ello genera "depósitos" de exceso de flujo en los vértices que va reduciendo a través de operaciones de push entre vértices concretos hasta alcanzar la factibilidad primal (y optimalidad).



Dinic intenta evitar la fase de construcción del camino aumentante aprovechando todos los caminos posibles de una longitud antes de pasar a la siguiente. 

El algoritmo queda como sigue:

<b>P1. Inicialización:</b> Inicializamos los flujos de los arcos que salen de origen a su máximo y el del resto de arcos a 0. Inicializamos la etiqueta del vértice inicio a |V| y el del resto de vértices a 0 

<b>P2. Mejora:</b> Mientras queden vértices activos (con overflow).<br>
$~~~$Sea v el vértice activo<br>
$~~~~~~$Si existe arco admisible, relabel(v)<br>
$~~~~~~$Si no push(arco)

In [ ]:
type graphGT
    n::Int64        # Number of nodes
    nArcos::Array{Int64,1} # número de arcos por vértice 
    adj::Array{Edge,2}  #adjacency
end

function fromSparseToGraphGT(g::gFlowsSparse)
    nArcos=zeros(Int64,g.nVertices)
    for i in 1:g.nArcos
        nArcos[g.arcos[i].origen] += 1
        nArcos[g.arcos[i].destino] += 1
    end
    adj= Array{Edge}(g.nVertices,maximum(nArcos))
    nArcos=zeros(Int64,g.nVertices)
    for i in 1:g.nArcos
        u=g.arcos[i].origen
        v=g.arcos[i].destino
        nArcos[u] += 1
        nArcos[v] += 1
        adj[u,nArcos[u]]=Edge(v,0,g.arcos[i].capacidad,nArcos[v])
        adj[v,nArcos[v]]=Edge(u,0,0,nArcos[u])
    end
    return graphGT(g.nVertices,nArcos,adj)
end


In [ ]:
#type Edge
#    v::Int64    # endpoint 
#    flow::Int64 # flow
#    c::Int64    # capacity
#    rev::Int64  # index of reverse edge
#end


function GTMaxFlow(g::graphGT,s::Int64,t::Int64)
    #inicialización
    maxflow=0
    label=zeros(Int64,g.n)
    label[s]=g.n
    exflow=PriorityQueue(Int64,Int64,Base.Order.Reverse)
    for i in 1:g.n
        enqueue!(exflow,i,0)
    end
    for i in 1:g.nArcos[s]
        exflow[g.adj[s,i].v]=g.adj[s,i].c
        g.adj[s,i].flow=g.adj[s,i].c
        g.adj[ g.adj[s,i].v , g.adj[s,i].rev ].flow = 0 - g.adj[s,i].c
    end
    node,value = peek(exflow)
    while value>0
        done=false
        #push
        # set γ=min{excess(node),u_f(e)} where v is the source of e and e is (u,v) and Ψ(u)=Ψ(v)+1
        # aument f along e by γ
        for c in 1:g.nArcos[node]
            if (done==false) && (label[node]>label[g.adj[node,c].v]) && (g.adj[node,c].flow<g.adj[node,c].c)
                #podemos hacer push de flow
                amount=min(value,g.adj[node,c].c-g.adj[node,c].flow)
                #println("push de flow entre ",node," y ",g.adj[node,c].v,"\t",amount)
                #actualizamos
                exflow[node] = value - amount
                if g.adj[node,c].v == t
                    maxflow += amount
                else
                    if g.adj[node,c].v != s
                        exflow[g.adj[node,c].v] += amount
                    end
                end
                g.adj[node,c].flow += amount
                g.adj[g.adj[node,c].v,g.adj[node,c].rev].flow -= amount
                done=true
            end
        end
        #relabel
        if done==false
        #set Ψ(v)= 1 + min{ ψ(w) : (v,w)∈δ^+_G_f(v)}
            minvalor=g.n*2
            for c in 1:g.nArcos[node]
                if (g.adj[node,c].flow<g.adj[node,c].c) && minvalor>label[g.adj[node,c].v]
                    minvalor=label[g.adj[node,c].v]
                end
            end
            minvalor += 1
            #println("update label ",node," de ",label[node]," a ",minvalor)
            if minvalor<=label[node]
                println("gazapo\n")
                return -1
            else
                label[node]=minvalor
            end
        end
        node,value = peek(exflow)
    end
    return maxflow
end

In [ ]:
gS=generarGrafoSparse(5000,0.1,1000)
gGT=fromSparseToGraphGT(gS)

@time(GTMaxFlow(gGT,1,2))
flow,pOrigen,destinos,edge,edgeOpposite=prepararSparse(gS)
@time(ffSparse(gS,flow,pOrigen,destinos,edge,edgeOpposite,1,2))

#maxFlow=GTMaxFlow(gGT,1,2)
#println("con GT: ",maxFlow)
#flow,pOrigen,destinos,edge,edgeOpposite=prepararSparse(gS)
#maxFlowS,flowsS=ffSparse(gS,flow,pOrigen,destinos,edge,edgeOpposite,1,2)
#println("flujo (3): ",maxFlowS)
